In [41]:
import warnings
warnings.simplefilter("ignore")

In [42]:
import datasets

from peft import PromptEncoderConfig, TaskType, get_peft_model

import opacus
from opacus.validators import ModuleValidator
from opacus import PrivacyEngine

import torch
import torch.nn as nn
import numpy as np

from tqdm.notebook import tqdm
from torch.optim import SGD
from torch.utils.data import DataLoader

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, DataCollatorWithPadding

from sklearn.metrics import accuracy_score

In [43]:
model_name = "prajjwal1/bert-tiny"
EPOCHS = 4
BATCH_SIZE = 32
LR = 5e-5

In [44]:
dataset = datasets.load_dataset('glue', 'sst2')
num_labels = dataset["train"].features["label"].num_classes

In [45]:
padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name)

if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    outputs = tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)
    return outputs

In [46]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence"],
)

tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [47]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [48]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], 
    shuffle=True, 
    batch_size=BATCH_SIZE, 
    collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_datasets["validation"], 
    batch_size=BATCH_SIZE, 
    collate_fn=data_collator
)

In [49]:
EPSILON = np.inf
DELTA = 1
MAX_GRAD_NORM = 0
MAX_PHYSICAL_BATCH_SIZE = int(BATCH_SIZE/4)

In [50]:
config = AutoConfig.from_pretrained(model_name)
config.num_labels = num_labels

peft_config = PromptEncoderConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=18,
    encoder_hidden_size=128
)

model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
model = get_peft_model(model, peft_config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-1): 2 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=128, out_features=128, bias=True)
                (key): Linear(in_features=128, out_features=128, bias=True)
                (value): Linear(in_features=128, out_features=128, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features

In [51]:
errors = ModuleValidator.validate(model, strict=False)
print(errors)

[]


In [52]:
optimizer = SGD(params=model.parameters(), lr=LR)

In [53]:
privacy_engine = PrivacyEngine(accountant="rdp")

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
    batch_first=True,
)

In [54]:
print(f"Using Sigma = {optimizer.noise_multiplier:.3f} | C = {optimizer.max_grad_norm} | Initial DP (ε, δ) = ({privacy_engine.get_epsilon(DELTA)}, {DELTA})")

Using Sigma = 10.000 | C = 0 | Initial DP (ε, δ) = (0, 1)


In [55]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable Parameters: {trainable_params} || All Parameters: {all_param} || Trainable Parameters (%): {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

Trainable Parameters: 52098 || All Parameters: 4438276 || Trainable Parameters (%): 1.17


In [56]:
def train(model, train_dataloader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    epsilon = []

    for i, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Training Epoch: {epoch}"):
        
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()

        outputs = model(**batch)
        loss = criterion(outputs.logits, batch["labels"])
        loss.backward()

        optimizer.step()
        losses.append(loss.item())

        if i % 64000 == 0:
            epsilon = privacy_engine.get_epsilon(DELTA)

            print(f"Training Epoch: {epoch} | Loss: {np.mean(losses):.6f} | ε = {epsilon:.2f}")

    return losses, epsilon

In [57]:
def test(model, test_dataloader, device):
    model.eval()
    eval_loss = 0
    eval_preds = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Test"):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
        loss = outputs.loss

        eval_loss += loss.detach().float()
        preds = outputs.logits.argmax(dim=-1)

        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )
        acc = accuracy_score(preds.cpu().numpy(), batch["labels"].cpu().numpy())

    print(
        f"Test set: Loss: {eval_loss:.4f}, Accuracy: {acc*100:.2f}%"
    )

    return eval_loss, acc

In [58]:
for epoch in tqdm(range(EPOCHS), desc=f'Training {EPOCHS} Epochs'):
    train(model, train_dataloader, optimizer, epoch + 1, device)

Training 4 Epochs:   0%|          | 0/4 [00:00<?, ?it/s]

Training Epoch: 1:   0%|          | 0/2105 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training Epoch: 1 | Loss: 0.702038 | ε = -3.35


Training Epoch: 2:   0%|          | 0/2105 [00:00<?, ?it/s]

Training Epoch: 2 | Loss: 0.704225 | ε = -3.35


Training Epoch: 3:   0%|          | 0/2105 [00:00<?, ?it/s]

Training Epoch: 3 | Loss: 0.695432 | ε = -3.35


Training Epoch: 4:   0%|          | 0/2105 [00:00<?, ?it/s]

Training Epoch: 4 | Loss: 0.682204 | ε = -3.35


In [59]:
final_epsilon = privacy_engine.get_epsilon(DELTA)
print(f"Final DP Guarantee (ε, δ)-DP = ({final_epsilon:.2f}, {DELTA})")

Final DP Guarantee (ε, δ)-DP = (-3.35, 1)


In [60]:
test(model, test_dataloader, device)

Test:   0%|          | 0/28 [00:00<?, ?it/s]

Test set: Loss: 0.7221, Accuracy: 50.00%


(tensor(0.7221, device='cuda:0'), 0.5)